<a href="https://colab.research.google.com/github/manugupta-or/Coursera_aiscm/blob/main/Vivo_supplier_selection_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Vivo Supplier Selection Problem

In [8]:
from ortools.linear_solver import pywraplp
import numpy as np
#storing data
discounts = [
    [7, 22, 25, 34, 35],
    [30, 18, 18, 31, 26],
    [21, 30, 34, 12, 14],
    [27, 29, 30, 60, 6],
    [31, 25, 10, 13, 30],
    [23, 32, 15, 60, 9],
    [6, 21, 18, 60, 28],
    [17, 18, 6, 8, 31]
]


demand = [592, 446, 548, 647, 245, 797, 603, 401]
list_price = [870, 630, 960, 400, 980, 510, 830, 550]
fixed_admin_cost = 50000
reliability = 0.8
num_product = len(discounts)
num_supplier = len(discounts[1])

# discounted_price = np.zeros(40).reshape(num_product,num_supplier)
# for i in range(num_product):
#   for j in range(num_supplier):
#     discounted_price[i][j] = (1-(discounts[i][j])/100) * list_price[i]

# print(discounted_price)

# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [9]:
#Create the decision variables 
infinity = solver.infinity()

y = {} #binary decision variable
for j in range(num_supplier):
    y[j] = solver.IntVar(0, 1, 'y[%i]' % (j+1))
print(y)

x_var = {} #Decision variable for the quantity of ith Product from jth Supplier 
k = 0
for i in range(num_product):
  x_var[i] = [solver.NumVar(0, infinity, 'X[%d][%d]' %((i+1),(j+1))) for j in range(num_supplier)]
 # k=k+1

print(x_var)
print(type(x_var))

import pandas as pd
pd_frame = pd.DataFrame.from_dict(x_var)
print(pd_frame.transpose())

print('Number of variables =', solver.NumVariables())

#      x_var[i] = [solver.IntVar(0, infinity, 'Supplier %d, Product %d' %((i+1),(y+1))) for y in range(len(data_arr['requirements']))]


{0: y[1], 1: y[2], 2: y[3], 3: y[4], 4: y[5]}
{0: [X[1][1], X[1][2], X[1][3], X[1][4], X[1][5]], 1: [X[2][1], X[2][2], X[2][3], X[2][4], X[2][5]], 2: [X[3][1], X[3][2], X[3][3], X[3][4], X[3][5]], 3: [X[4][1], X[4][2], X[4][3], X[4][4], X[4][5]], 4: [X[5][1], X[5][2], X[5][3], X[5][4], X[5][5]], 5: [X[6][1], X[6][2], X[6][3], X[6][4], X[6][5]], 6: [X[7][1], X[7][2], X[7][3], X[7][4], X[7][5]], 7: [X[8][1], X[8][2], X[8][3], X[8][4], X[8][5]]}
<class 'dict'>
         0        1        2        3        4
0  X[1][1]  X[1][2]  X[1][3]  X[1][4]  X[1][5]
1  X[2][1]  X[2][2]  X[2][3]  X[2][4]  X[2][5]
2  X[3][1]  X[3][2]  X[3][3]  X[3][4]  X[3][5]
3  X[4][1]  X[4][2]  X[4][3]  X[4][4]  X[4][5]
4  X[5][1]  X[5][2]  X[5][3]  X[5][4]  X[5][5]
5  X[6][1]  X[6][2]  X[6][3]  X[6][4]  X[6][5]
6  X[7][1]  X[7][2]  X[7][3]  X[7][4]  X[7][5]
7  X[8][1]  X[8][2]  X[8][3]  X[8][4]  X[8][5]
Number of variables = 45


In [10]:
#Create the constraints

# Creating reliability constraint
for i in range(num_product):
  for j in range(num_supplier):
    solver.Add(x_var[i][j] <= reliability*demand[i]*y[j])
    #p#rint(x_var[i][j])
    #print(demand[i])
   # print(y[j])

#Creating demand satisfaction constraint
for i in range(num_product):
  expr = [x_var[i][j] for j in range(num_supplier)]
  solver.Add(sum(expr) >= demand[i])
  #print(expr)
  #print(sum(expr))
  #print(demand[i])

print('Number of constraints =', solver.NumConstraints())


Number of constraints = 48


In [11]:

objective_terms = []
for i in range(num_product):
    for j in range(num_supplier):
        objective_terms.append((1-(discounts[i][j])/100) * list_price[i] * x_var[i][j])

for j in range(num_supplier):
    objective_terms.append(fixed_admin_cost  * y[j])


solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

In [19]:
if status == pywraplp.Solver.OPTIMAL:
  print('Minimum Cost Incurred to Satisfy Demand = ', solver.Objective().Value())
  print()
  for j in range(num_supplier):
    print(y[j] , ' = ', y[j].solution_value())
  for i in range(num_product):
    print()
    for j in range(num_supplier):
      if(round(x_var[i][j].solution_value(),2) == -0.0):
        print(x_var[i][j], " = ", 0.0 , "| ",end=" ")
      else:
        print(x_var[i][j], " = ", round(x_var[i][j].solution_value(),2), "| ",end=" ")  

else:
        print('The problem does not have an optimal solution.')


Minimum Cost Incurred to Satisfy Demand =  1933110.7200000002

y[1]  =  0.0
y[2]  =  1.0
y[3]  =  -0.0
y[4]  =  1.0
y[5]  =  1.0

X[1][1]  =  0.0 |  X[1][2]  =  0.0 |  X[1][3]  =  0.0 |  X[1][4]  =  118.4 |  X[1][5]  =  473.6 |  
X[2][1]  =  0.0 |  X[2][2]  =  0.0 |  X[2][3]  =  0.0 |  X[2][4]  =  356.8 |  X[2][5]  =  89.2 |  
X[3][1]  =  0.0 |  X[3][2]  =  438.4 |  X[3][3]  =  0.0 |  X[3][4]  =  0.0 |  X[3][5]  =  109.6 |  
X[4][1]  =  0.0 |  X[4][2]  =  129.4 |  X[4][3]  =  0.0 |  X[4][4]  =  517.6 |  X[4][5]  =  0.0 |  
X[5][1]  =  0.0 |  X[5][2]  =  49.0 |  X[5][3]  =  0.0 |  X[5][4]  =  0.0 |  X[5][5]  =  196.0 |  
X[6][1]  =  0.0 |  X[6][2]  =  159.4 |  X[6][3]  =  0.0 |  X[6][4]  =  637.6 |  X[6][5]  =  0.0 |  
X[7][1]  =  0.0 |  X[7][2]  =  0.0 |  X[7][3]  =  0.0 |  X[7][4]  =  482.4 |  X[7][5]  =  120.6 |  
X[8][1]  =  0.0 |  X[8][2]  =  80.2 |  X[8][3]  =  0.0 |  X[8][4]  =  0.0 |  X[8][5]  =  320.8 |  